In [96]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests as gc_test
import random
import itertools
from scipy.stats import ttest_ind

In [112]:
train =pd.read_pickle("model_datasets//train_model.pbz2")

# Teste de Causaliade de Granger

In [ ]:
train_gc=train.set_index(pd.DatetimeIndex(train.date))

In [ ]:
train_gc=train_gc[['fips','score']]
train_gc=train_gc.pivot(columns= 'fips',values='score')


In [45]:
gc_test(train_gc,maxlag=6)


Granger Causality
number of lags (no zero) 1


InfeasibleTestError: The Granger causality test statistic cannot be compute because the VAR has a perfect fit of the data.

O teste está dando erro pois algumas das séries são exatamente igais as outras, isso provavelemtne se da deviso a regiões muito próximas terem climas iguais ou muito parecidos, como o objetivo deste teste é mostrar que as condições climáticas se movem ao longo do tempo, irei fazer uma amostragem aleatória de  fips de diferentes estados para verificar 

In [61]:
fips=list(train_gc.columns)
fips_sample=random.sample(fips,100)
testes =[]
for pair in itertools.permutations(fips_sample,2):
    pair=list(pair)
    testes.append(gc_test(train_gc[pair],maxlag=6,verbose=False))

In [64]:
testes[:5]

[{1: ({'ssr_ftest': (14.738648495028487, 0.0001323001173219633, 883.0, 1),
    'ssr_chi2test': (14.788723178477053, 0.0001202524611392519, 1),
    'lrtest': (14.666656192811388, 0.00012829589109092726, 1),
    'params_ftest': (14.738648495028396, 0.000132300117321968, 883.0, 1.0)},
    array([[0., 1., 0.]])]),
  2: ({'ssr_ftest': (7.227253381408026, 0.000770447110514635, 880.0, 2),
    'ssr_chi2test': (14.536634642150233, 0.0006972843127828076, 2),
    'lrtest': (14.41853977394976, 0.0007396970207870043, 2),
    'params_ftest': (7.227253381407915, 0.0007704471105147371, 880.0, 2.0)},
    array([[0., 0., 1., 0., 0.],
           [0., 0., 0., 1., 0.]])]),
  3: ({'ssr_ftest': (7.518546548232074, 5.6954630501008584e-05, 877.0, 3),
    'ssr_chi2test': (22.73567325645548, 4.584391459994654e-05, 3),
    'lrtest': (22.44822112288466, 5.262053813200623e-05, 3),
    'params_ftest': (7.5185465482320275, 5.695463050101698e-05, 877.0, 3.0)},
    array([[0., 0., 0., 1., 0., 0., 0.],
           [0., 0

In [65]:
fips_sample[:6]

['31147', '8053', '19035', '31181', '48089', '19113']

# Teste de hipoteses quanto a semelhança de distribuiçoes

Teste das dsitirbuições de QV2M

In [117]:
qv2m =train.groupby('score')['QV2M']
qv2m_dic={group: qv2m.get_group(group) for group in qv2m.groups}
qv2m_test={key: ttest_ind(qv2m_dic[5],qv2m_dic[key],equal_var=False) for key in [0,1,2,3,4]}
qv2m_test

{0: Ttest_indResult(statistic=-1.2641925112729082, pvalue=0.20617487422124484),
 1: Ttest_indResult(statistic=24.960856309144038, pvalue=8.746656114642769e-136),
 2: Ttest_indResult(statistic=22.53193910083926, pvalue=2.3631993390207524e-111),
 3: Ttest_indResult(statistic=25.60964509530738, pvalue=5.31374236616931e-143),
 4: Ttest_indResult(statistic=26.156230264789375, pvalue=2.4395007251807277e-149)}

Teste mostra qeu todas disitribuiçõesde QV2M menos a classificação normal é semlhante a classificação de seca extrema

In [118]:
t2m=train.groupby('score')['T2M']
t2m_dic={group: t2m.get_group(group) for group in t2m.groups}
t2m_test={key: ttest_ind(t2m_dic[5],t2m_dic[key],equal_var=False) for key in [0,1,2,3,4]}
t2m_test

{0: Ttest_indResult(statistic=77.66730466689026, pvalue=0.0),
 1: Ttest_indResult(statistic=80.38520905488554, pvalue=0.0),
 2: Ttest_indResult(statistic=66.44628873314828, pvalue=0.0),
 3: Ttest_indResult(statistic=55.83434762221066, pvalue=0.0),
 4: Ttest_indResult(statistic=45.46959373655769, pvalue=0.0)}

In [121]:
ts=train.groupby('score')['TS']
ts_dic={group: ts.get_group(group) for group in ts.groups}
ts_test={key: ttest_ind(ts_dic[5],ts_dic[key],equal_var=False) for key in [0,1,2,3,4]}
ts_test

{0: Ttest_indResult(statistic=81.82589777258245, pvalue=0.0),
 1: Ttest_indResult(statistic=82.86349067906018, pvalue=0.0),
 2: Ttest_indResult(statistic=68.06721081943995, pvalue=0.0),
 3: Ttest_indResult(statistic=56.904264593361546, pvalue=0.0),
 4: Ttest_indResult(statistic=46.24243022006639, pvalue=0.0)}